## Loading credentials from another config file

In [1]:
import config
import pandas as pd
import requests
import spotipy
import pickle
from sklearn.preprocessing import StandardScaler
import json
from IPython.display import IFrame

## Initializing Spotify API

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

## Loading the clustered data frame and the respective model

In [3]:
#Loading the clustered data frame and the respective model

clustered_df = pd.read_csv('Data/lyric_features_b.csv')

with open("Model/lyric_features_b.pkl", "rb") as f:
    scaler, kmeans = pickle.load(f)


In [4]:
len(clustered_df['cluster'].unique())

7

In [5]:
kmeans.inertia_

22530.74489996135

## Defining functions

The recommender consists of 3 steps:

1. search(): it lets the user serch a song using sp.search from Spotipy. The top 10 results are presented, and the user is asked to identify the correct song.
2. recommend (): the function extracts the audio features from the song selected by the user. It then predicts in which cluster the songs belongs, and provides a random song of the same cluster from the database. 
3. The resulting recommendation is displayed with an embedded player, using the IFrame library.

In [7]:
def search():
    # Get user input for song name
    song_name = input("Enter the name of a song: ")

    # Use the Spotify search API to find songs matching the user input
    results = sp.search(q=song_name, type='track')

    # Display the top matching songs to the user
    for i, track in enumerate(results['tracks']['items'][:10]):
        print(f"{i+1}. {track['name']} by {track['artists'][0]['name']}")

    # Ask the user to select the song they meant from the list
    selected_song = None
    while not selected_song:
        selection = input("Please select the song you meant by number (or type 'exit' to cancel): ")
        if selection == 'exit':
            return
        try:
            selected_song = results['tracks']['items'][int(selection)-1]
        except:
            print("Invalid selection, please try again.")

    # Get the track ID of the selected song
    track_id = selected_song['id']

    # Display the selected song using an embedded player
    print(f"Now playing: {selected_song['name']} by {selected_song['artists'][0]['name']}")
    display(play(track_id))

    return(track_id)

In [8]:
def recommend(track_id):
    # Use the Spotify API to get the audio features for the selected song
    audio_features = sp.audio_features(track_id)[0]

    # Scale the audio features using the same scaler used to fit the KMeans model
    X = pd.DataFrame(audio_features, index=[0], columns=clustered_df.drop(columns=['id', 'cluster']).columns.tolist())
    X_scaled = scaler.transform(X)

    # Use the KMeans model to predict the cluster for the selected song
    cluster = kmeans.predict(X_scaled)[0]

    # Filter for songs in the same cluster as the selected song
    recommended_songs = clustered_df[clustered_df['cluster'] == cluster]

    # Recommend a random song from the list of songs in the same cluster as the selected song
    recommended_song_id = recommended_songs.sample()['id'].values[0]

    # Get the recommended song name
    recommended_song_name = sp.track(recommended_song_id)['name']
    recommended_artist_name = sp.track(recommended_song_id)['artists'][0]['name']

    
    # Display the recommended song using an embedded player
    print(f"We recommend '{recommended_song_name}' by {recommended_artist_name} based on your selection of '{sp.track(track_id)['name']}'")
    display(play(recommended_song_id))

    # Ask the user if they want more recommendations
    while True:
        choice = input("Would you like more recommendations? (y/n) ")
        if choice == "y":
            recommend(track_id)
            break
        elif choice == "n":
            break
        else:
            print("Invalid input. Please enter 'y' or 'n'.")

In [6]:
def play(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

## Running the code

Let's experiment and evaluate the responses. 

In [9]:
recommend(search())

Enter the name of a song: diamonds
1. Diamonds by Rihanna
2. Diamonds by Chris Travis
3. Diamonds & Gasoline by Turnpike Troubadours
4. Bubblegum Bitch by MARINA
5. Diamonds From Sierra Leone - Remix by Kanye West
6. Diamonds Dancing (feat. Travis Scott) by Young Stoner Life
7. Diamonds by Sam Smith
8. Ghost by Ava Max
9. Diamonds - Remix by Rihanna
10. Diamonds by Bas
Please select the song you meant by number (or type 'exit' to cancel): 1
Now playing: Diamonds by Rihanna


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


We recommend 'Real Love' by Dillon Francis based on your selection of 'Diamonds'


Would you like more recommendations? (y/n) y


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


We recommend 'Lionheart (Fearless)' by Joel Corry based on your selection of 'Diamonds'


Would you like more recommendations? (y/n) y


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


We recommend 'Closer' by Aevion based on your selection of 'Diamonds'


Would you like more recommendations? (y/n) n
